In [5]:
# change to store dataset
STORE_VHB = True
STORE_SCIMAGO = True

In [6]:
import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession

from bs4 import BeautifulSoup
import time
import csv

In [7]:
# get current year to save in applicable folder
current_year = pd.datetime.now().year

In [8]:
def get_source(url):
    """Return the source code for the provided URL. 
    Args: 
        url (string): URL of the page to scrape.
    Returns:
        response (object): HTTP response object from requests_html. 
    """
    time.sleep(5) # prevents captcha hopefully

    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)

# VHB

In [9]:
# get website content
response = get_source("https://vhbonline.org/vhb4you/vhb-jourqual/vhb-jourqual-3/gesamtliste")
soup = BeautifulSoup(response.html.raw_html)

In [10]:
# get table content
table = soup.find("tbody")
table_content = table.find_all("tr")

journal_titles = []
issn_numbers = []
jq3_ratings = []
in_partial_rankings = []

# loop over table and add to according list
for table_line in table_content:
    columns = table_line.find_all("td")
    
    journal_titles.append(columns[0].text.strip())
    issn_numbers.append(columns[1].text.strip())
    jq3_ratings.append(columns[2].text.strip())
    in_partial_rankings.append(columns[3].text.strip())

In [11]:
# create dataframe
data = {'JOURNAL': journal_titles, 
        'ISSN': issn_numbers, 
        'JQ3': jq3_ratings,  
        'IN_PARTIAL_RANKING': in_partial_rankings
       }

journal_ratings_from_vhb = pd.DataFrame(data)

In [12]:
if STORE_VHB:
    journal_ratings_from_vhb.to_csv(f'../data/{current_year}/journal_ratings_vhb.csv')

# Scimago

In [13]:
# get first page content
response = get_source("https://www.scimagojr.com/journalrank.php?out=xls")

if STORE_SCIMAGO:
    csv_file = open(f'../data/{current_year}/journal_ratings_scimago.csv', 'wb')
    csv_file.write(response.content)
    csv_file.close()


In [14]:
pd.read_csv(f'../data/{current_year}/journal_ratings_scimago.csv', sep=";").head(5)

C:\Users\katma\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Rank,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (2020),Total Docs. (3years),Total Refs.,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories
0,1,28773,Ca-A Cancer Journal for Clinicians,journal,"15424863, 00079235","62,937",Q1,168,47,119,3452,15499,80,"126,34","73,45",United States,Northern America,Wiley-Blackwell,1950-2020,Hematology (Q1); Oncology (Q1)
1,2,19434,MMWR Recommendations and Reports,journal,"10575987, 15458601","40,949",Q1,143,10,9,1292,492,9,"50,00","129,20",United States,Northern America,Centers for Disease Control and Prevention (CDC),1990-2020,Epidemiology (Q1); Health Information Manageme...
2,3,20315,Nature Reviews Molecular Cell Biology,journal,"14710072, 14710080","37,461",Q1,431,115,338,8439,10844,167,"32,83","73,38",United Kingdom,Western Europe,Nature Publishing Group,2000-2020,Cell Biology (Q1); Molecular Biology (Q1)
3,4,29431,Quarterly Journal of Economics,journal,"00335533, 15314650","34,573",Q1,259,40,110,2733,1945,109,"16,00","68,33",United Kingdom,Western Europe,Oxford University Press,1886-2020,Economics and Econometrics (Q1)
4,5,21100812243,Nature Reviews Materials,journal,20588437,"32,011",Q1,108,92,264,10632,11188,138,"32,15","115,57",United Kingdom,Western Europe,Nature Publishing Group,2016-2020,"Biomaterials (Q1); Electronic, Optical and Mag..."
